In [2]:
%matplotlib widget
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)

from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [3]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"
path_figs = os.getcwd() +"\\..\\Figures\\" 

In [85]:
# Find number of citizens in region
latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir
df_kommunekort = pd.read_csv(latestdir+'/Kommunalt_DB/10_Kommune_kort.csv',encoding='latin1',
                                delimiter = ';')
df_kommunekort = df_kommunekort.set_index("Kommunenavn")

# Get data

Since data only contains the most recent numbers, go through every directory to generate time-series

In [150]:
allSubDirs = list(os.walk(path_dash))[0][1]

df = pd.DataFrame()

for curSubDir in allSubDirs:
    curdir = path_dash + curSubDir
    curfilepath = curdir+'/Kommunalt_DB/17_tilfaelde_fnkt_alder_kommuner.csv'

    # Check if file was included at the time. The "Kommune/17" file wasn't included until 2021-09-22 
    if os.path.isfile(curfilepath):
        curdf = pd.read_csv(curfilepath,encoding='latin1',delimiter = ';')
        
        df = pd.concat([df,curdf])

# Set dtypes
df.Kommune = df.Kommune.fillna(0)  # All NaN kommuner is set to zero
df['Kommune'] = df['Kommune'].astype(int)
df['Dagsdato'] = pd.to_datetime(df['Dagsdato'])
df['Bekræftede tilfælde'] = pd.to_numeric(df['Bekræftede tilfælde'])
df['Aldersgruppe'] = df.Aldersgruppe.replace('00-02','0-2')
df['Aldersgruppe'] = df.Aldersgruppe.replace('03-05','3-5')
df['Aldersgruppe'] = df.Aldersgruppe.replace('06-11','6-11')

# df['Forskel'] = df['Bekræftede tilfælde'].diff().fillna(0).astype(int,errors='ignore')

In [165]:
# np.array(dayDiff).unique()
# pd.DataFrame(data=dayDiff).loc[:,0].unique()

array([0, 1, 3], dtype=int64)

In [176]:

# curKom = 101
# curAge = '6-11'
# curdf = df[(df.Kommune == curKom) & (df.Aldersgruppe == curAge)]
# curdf.tail(10)
# # df[df.Kommune == 101].tail(20)

def getDiffTimeSeries(komCode,Age):

    curdf = df[(df.Kommune == komCode) & (df.Aldersgruppe == Age)] 

        
    dayDiff = [int(x) for x in (curdf.Dagsdato.diff()/np.timedelta64(1,'D')).fillna(0)]

    curDays = []
    curDiffs = []
    for i in range(1,len(curdf)):
        curRow = curdf.iloc[i]
        prevRow = curdf.iloc[i-1]
        # print(curRow)
        if (dayDiff[i] == 1):
            curDays.append(curRow.Dagsdato)
            curDiffs.append(curRow['Bekræftede tilfælde']-prevRow['Bekræftede tilfælde']) 
        elif (dayDiff[i] == 3):
            curCount = curRow['Bekræftede tilfælde']-prevRow['Bekræftede tilfælde']
            curDays.append(curRow.Dagsdato-np.timedelta64(2,'D'))
            curDays.append(curRow.Dagsdato-np.timedelta64(1,'D'))
            curDays.append(curRow.Dagsdato)
            curDiffs.append(curCount/3) 
            curDiffs.append(curCount/3) 
            curDiffs.append(curCount/3) 

    return np.array(curDays),np.array(curDiffs)

curDates,curCounts = getDiffTimeSeries(101,'6-11')

# print(curDates[-10:])
# print(curCounts[-10:])
# plt.figure()
# plt.plot(curDays,curDiffs)
# plt.plot(rnTime(curDays,7),rnMean(curDiffs,7))

In [200]:
allAge = df.Aldersgruppe.unique()[:-1]

fig,ax1 = plt.subplots()

import matplotlib
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","violet","blue","green"],len(allAge))

curKom = 101
# for curAge in allAge:
for i in range(0,len(allAge)):
    curAge = allAge[i]
    curColor = cmap(i)
    curDates,curCounts = getDiffTimeSeries(101,curAge)
    ax1.plot(curDates,curCounts,'.:',linewidth=0.5,markersize=2,color=curColor)
    ax1.plot(rnTime(curDates,7),rnMean(curCounts,7),label=curAge,color=curColor)
ax1.set_ylim(bottom=0)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))

ax1.set_xlim()
ax1.set_xlim(left=np.datetime64('2021-11-01'))
# Draw weekends
firstSunday = np.datetime64('2021-10-03')
numWeeks = 52
for k in range(-numWeeks,numWeeks):
        curSunday = firstSunday + np.timedelta64(7*k,'D')
        ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
ax1.grid(axis='y')
ax1.legend()
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [114]:
# df.groupby(['Kommune','Aldersgruppe'])
# asdf = df.set_index(['Kommune','Aldersgruppe'])
df.columns

curKom = 101
curAge = '6-11'
asdf = df[(df.Kommune == curKom) & (df.Aldersgruppe == curAge)]

plt.figure()
plt.plot(asdf.Dagsdato,asdf['Bekræftede tilfælde'].diff(),'*')
plt.plot(rnTime(asdf.Dagsdato,7),rnMean(asdf['Bekræftede tilfælde'].diff(),7))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …